# 如何优雅高效的撸数据？

Tushare的行情等时间序列数据，一般都有两个常用参数：trade_date和ts_code，分别是交易日期和证券代码。如果你是想提取部分个股的历史数据，用ts_code参数，加上开始和结束日期可以方便提取数据。

但！如果是要获取所有历史数据，我们不建议通过ts_code来循环，而是用trade_date来提取，道理很简单，股票有3800多个，需要循环3800多次，每年的交易日也就才220左右，所以效率更高。总的来说，积分越高可以调取的频次会越高。

也就是以下方式：

In [1]:
import tushare as ts
import time

ts.set_token('60348c9924c35ceb20e8fa05df697f7304537b511b45def253d713c3')


pro = ts.pro_api()

df = pro.daily(trade_date='20200325')
print(df.head())

     ts_code trade_date   open   high    low  close  pre_close  change  \
0  002507.SZ   20200325  30.79  31.07  29.11  29.90      30.11   -0.21   
1  603366.SH   20200325   5.68   5.74   5.57   5.65       5.55    0.10   
2  603369.SH   20200325  27.11  28.06  26.86  27.72      26.43    1.29   
3  002519.SZ   20200325   4.82   4.84   4.62   4.68       4.69   -0.01   
4  603389.SH   20200325   4.50   4.54   4.43   4.53       4.43    0.10   

   pct_chg        vol      amount  
0  -0.6974  263059.08  783721.546  
1   1.8018  129442.82   73156.302  
2   4.8808  134490.08  368846.960  
3  -0.2132  335282.74  158588.520  
4   2.2573   27885.80   12520.168  


在循环提取数据时，首先我们可以通过交易日历拿到一段历史的交易日。

In [17]:
#获取20200101～20200401之间所有有交易的日期
dt = pro.trade_cal(exchange='SSE', is_open='1', 
                            start_date='20200325', 
                            end_date='20200328', 
                            fields='cal_date')

print(dt.head())

   cal_date
0  20200327
1  20200326
2  20200325


循环过程中，为了保持数据提取的稳定性，可以先建立一个专门的函数，实现一个重试机制：

In [24]:
def get_daily(trade_date='', start_date='', end_date=''):
    for _ in range(3):
        try:
            if trade_date:
                df = pro.daily(trade_date=trade_date)
                print(1)
            else:
                df = pro.daily(start_date=start_date, end_date=end_date)
                print(2)
        except:
            time.sleep(1)
        else:
            print(1)
            print(df.head())
            return df

然后通过在循环中调取数据：

In [25]:
for date in dt['cal_date'].values:
    print(date)
    df = get_daily(date)

20200327
1
1
     ts_code trade_date   open   high    low  close  pre_close  change  \
0  600740.SH   20200327   5.44   5.45   5.35   5.35       5.37   -0.02   
1  600750.SH   20200327  11.86  11.89  11.65  11.66      11.78   -0.12   
2  600757.SH   20200327   5.42   5.48   5.39   5.39       5.38    0.01   
3  600760.SH   20200327  27.81  28.40  27.63  27.98      27.56    0.42   
4  600769.SH   20200327   4.92   5.03   4.91   5.01       4.92    0.09   

   pct_chg       vol      amount  
0  -0.3724  60705.25   32748.994  
1  -1.0187  44281.16   52112.392  
2   0.1859  62069.00   33649.513  
3   1.5239  97228.22  272089.201  
4   1.8293  43928.57   21867.194  
20200326
1
1
     ts_code trade_date    open    high     low   close  pre_close  change  \
0  002891.SZ   20200326   21.35   21.82   21.13   21.48      21.35    0.13   
1  002903.SZ   20200326   12.83   13.11   12.48   12.54      12.74   -0.20   
2  002912.SZ   20200326  157.10  160.92  155.50  158.06     159.73   -1.67   
3  0029

In [5]:
df

error:vs code output exceeds the size limit.
settings
"notebook.output.textLineLimit": 500